In [11]:
import logging
import os
import pickle
import time
import statistics
from abc import abstractmethod

import colorsys
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import hdbscan

from sklearn.cluster import OPTICS, cluster_optics_dbscan
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.metrics import silhouette_score, silhouette_samples
import statistics

from sklearn.neighbors import LocalOutlierFactor

from sklearn.svm import OneClassSVM




In [12]:
f0 = 'no anomaly.csv'
f1 = 'engine.csv'
f2 = 'elevator.csv'
f3 = 'aileron.csv'
f4 = 'rudder.csv'

base_df = pd.read_csv(f0, header=0, index_col=None)
engine1_df = pd.read_csv(f1, header=0, index_col=None)
elevator1_df = pd.read_csv(f2, header=0, index_col=None)
aileron1_df = pd.read_csv(f3, header=0, index_col=None)
rudder1_df = pd.read_csv(f4, header=0, index_col=None)

df_list = [base_df, engine1_df, elevator1_df, aileron1_df, rudder1_df]

for df in df_list:

    df['airspeedchange'] = df['Airspeed']- df['Airspeed'].shift(1)
    df['airspeedchange'].fillna(0, inplace=True)

anomaly_list = aileron1_df['Anomaly'].tolist()

base_df = base_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
clustering_engine_df = engine1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
clustering_elevator1_df = elevator1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
clustering_aileron1_df = aileron1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]
clustering_rudder_df = rudder1_df[['Roll', 'Pitch', 'Yaw', 'Airspeed', 'VelocityX', 'VelocityY', 'VelocityZ', 'airspeedchange']]

In [13]:
anomaly_list.count(1)

2349

In [14]:
#train the 5 models

kmeans = KMeans(init="random", n_clusters=2, n_init=10, random_state=1)
kmeans.fit(base_df)

lof = LocalOutlierFactor(n_neighbors=14, contamination="auto",novelty=True)
lof.fit(base_df)

nu = 0.1
kernel = 'rbf'
gamma = 'scale'

model = OneClassSVM(nu=nu, kernel=kernel, gamma=gamma)
model.fit(base_df)

db = DBSCAN(eps = 0.78).fit(base_df)

optics_clustering = OPTICS(min_samples = 50).fit(base_df)



In [15]:
def rule_checker(datapoint):
    rules_broken = 0
    if datapoint["Roll"] > 47.3 or datapoint["Roll"] < -50:
        rules_broken += 1

    if datapoint["Pitch"] > 22.93 or datapoint["Pitch"] < -19.3:
        rules_broken += 1

    if datapoint["Yaw"] > 179.2 or datapoint["Yaw"] < -177.3:
        rules_broken += 1

    if datapoint["Airspeed"] > 24.29 or datapoint["Airspeed"] < 12.48:
        rules_broken += 1

    if datapoint["VelocityX"] > 19.73 or datapoint["VelocityX"] < -17.05:
        rules_broken += 1

    if datapoint["VelocityY"] > 21.4 or datapoint["VelocityY"] < -18.26:
        rules_broken += 1

    if datapoint["VelocityZ"] > 5.3 or datapoint["VelocityZ"] < -8.6:
        rules_broken += 1

    return rules_broken

<h3>aileron<h3>

In [16]:
aileron_rule_alerts = []

for i in range (len(aileron1_df)):
    alerts = 0

    alerts += (rule_checker(aileron1_df.iloc[i]))
    
    aileron_rule_alerts.append(alerts)
    
count = 0
for value in aileron_rule_alerts:
    if value >0:
        count +=1


#predict for aileron

aileron_kmeans = []
aileron_lof = []
aileron_svm = []
aileron_db = []
aileron_optics = []

base_df2 = base_df.copy()
base_df2['cluster'] = kmeans.labels_
aileron1_df2 = clustering_aileron1_df.copy()
aileron1_df2['cluster'] = kmeans.predict(clustering_aileron1_df)

centroids = kmeans.cluster_centers_

original_distances = []
for i in range(len(base_df)):
    centroid = centroids[int(base_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((base_df.iloc[i] - centroid)**2))
    original_distances.append(distance)

original_mean = sum(original_distances)/len(original_distances)

original_std = np.std(original_distances)

threshold = original_mean + 3* original_std

for i in range(len(clustering_aileron1_df)):
    centroid = centroids[int(aileron1_df2.iloc[i]['cluster'])]
    distance = np.sqrt(np.sum((clustering_aileron1_df.iloc[i] - centroid)**2))
    if distance > threshold:
        aileron_kmeans.append(-1)
    else:
        aileron_kmeans.append(1)


for i in range(clustering_aileron1_df.shape[0]):
    prediction = lof.predict(clustering_aileron1_df.iloc[[i]])
    for value in prediction:
        aileron_lof.append(value)

for i in range(clustering_aileron1_df.shape[0]):
    prediction = model.predict(clustering_aileron1_df.iloc[[i]])
    for value in prediction:
        aileron_svm.append(value)

for value in db.fit_predict(clustering_aileron1_df):
    aileron_db.append(value)

for value in optics_clustering.fit_predict(clustering_aileron1_df):
    aileron_optics.append(value)


#get predictions for aileron using voting

final_aileron_predictions = []

for i in range (len(aileron1_df)):
    predictions = [aileron_kmeans[i], aileron_lof[i], aileron_svm[i], aileron_db[i], aileron_optics[i]]
    final_aileron_predictions.append(max(set(predictions), key=predictions.count))
    
overall_aileron_count = 0

for i in range(len(aileron1_df)):
    if final_aileron_predictions[i] <0 or aileron_rule_alerts[i] > 0:
        overall_aileron_count+=1

print("For aileron, out of the", len(aileron1_df), "anomalies", overall_aileron_count ,"were detected, which is", overall_aileron_count/len(aileron1_df)*100,"%")


ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
unsupervised_count = 0

for i in range(len(aileron1_df)):
    if final_aileron_predictions[i] < 0:
        unsupervised_count += 1

print(unsupervised_count)


2194


In [ ]:
rule_count = 0

for i in range(len(aileron1_df)):
    if aileron_rule_alerts[i] > 0:
        rule_count += 1

print(rule_count)

2506
